# Capstone Project- Data Clean and Process Workbook
Cary Mosley, May 2020

## Import Libraries

In [5]:
import pandas as pd
import calendar, time
import datetime as dt
from datetime import date
from pandas.tseries.offsets import DateOffset
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize
import string, re 

import warnings
warnings.filterwarnings("once")

pd.reset_option('max_rows')

## Data Clean

### Clean NYT Article Data

In [6]:
#Set Years
years = range(2005,2021)

#Select keywords for articles
keywords = ['Stock','Market','Finance','Business','Price','Debt','Portfolio','SP500',
            'Nasdaq','Dow Jones']

#Create DataFrame
nyt_df_final = pd.DataFrame(columns=['Date', 'Headline','Snippet'])

#Select just the headlines and snippets containing keywords
for year in years:
    
    try:
        #read in csv data
        path = '../Data/nyt'+str(year)+'1.csv'
        nyt_df = pd.read_csv(path,index_col=None)
        nyt_df.drop(columns=['Unnamed: 0'],inplace=True)
        
        #select date and convert to datetime object
        nyt_df['Date']=nyt_df['Date'].apply(lambda x: x[0:10])
        nyt_df['Date']=pd.to_datetime(nyt_df['Date'])
        
        #all headlines are there but some snippets are missing
        nyt_df.dropna(inplace=True)
        
        #select headlines and snippets containing the keywords
        nyt_df_headlines = nyt_df[nyt_df['Headline'].str.contains('|'.join(keywords))]
        nyt_df_headlines = nyt_df_headlines[~nyt_df_headlines["Headline"].str.contains('Corrections')]
        nyt_df_snippets = nyt_df[nyt_df['Snippet'].str.contains('|'.join(keywords))]
        
        #concatenate dataframes
        nyt_df_final = pd.concat([nyt_df_final,nyt_df_headlines,nyt_df_snippets])
        print(year,'1')
    except:
        print('error',year,'1')
        continue
    
    try:
        #read in csv data
        path = '../Data/nyt'+str(year)+'2.csv'
        nyt_df = pd.read_csv(path,index_col=None)
        nyt_df.drop(columns=['Unnamed: 0'],inplace=True)
        
        #select date and convert to datetime object
        nyt_df['Date']=nyt_df['Date'].apply(lambda x: x[0:10])
        nyt_df['Date']=pd.to_datetime(nyt_df['Date'])
        
        #all headlines are there but some snippets are missing
        nyt_df.dropna(inplace=True)
        
        #select headlines and snippets containing the keywords
        nyt_df_headlines = nyt_df[nyt_df['Headline'].str.contains('|'.join(keywords))]
        nyt_df_headlines = nyt_df_headlines[~nyt_df_headlines["Headline"].str.contains('Corrections')]
        nyt_df_snippets = nyt_df[nyt_df['Snippet'].str.contains('|'.join(keywords))]
        
        #concatenate dataframes
        nyt_df_final = pd.concat([nyt_df_final,nyt_df_headlines,nyt_df_snippets])
        print(year,'2')
    except:
        print('error',year,'2')
        continue

#Set the nyt_df_final index as the Date
nyt_df_final.set_index('Date', inplace=True)
nyt_df_final.drop_duplicates(inplace=True)
#Write to pickle
nyt_df_final.to_pickle('../Data/nyt_df')

error 2005 1
2006 1
error 2006 2
2007 1
2007 2
2008 1
2008 2
2009 1
2009 2
2010 1
2010 2
2011 1
2011 2
2012 1
2012 2
2013 1
2013 2
2014 1
2014 2
2015 1
2015 2
2016 1
2016 2
2017 1
2017 2
2018 1
2018 2
2019 1
2019 2
2020 1
error 2020 2


In [7]:
nyt_df_final.shape

(34930, 2)

In [8]:
nyt_df_final.head()

,Headline,Snippet
Date,,
2006-06-01,Glass Makers Fined for Price Fixing,The European Commission issued one of its larg...
2006-06-01,2 New Captains of the Economy Face Volatile Gl...,One reached the pinnacle of wealth and prestig...
2006-06-01,"For Google, M&A is Bad Business",Google on Wednesday poured cold water on specu...
2006-06-01,Nasdaq's Greifeld Plays the Tortoise in L.S.E....,"Nasdaq's chief executive, Robert Greifeld, is ..."
2006-06-01,Glass Makers Fined for Price Fixing,The European Commission issued one of its larg...


### Clean Ticker Data

In [9]:
#Import Ticker Data
spy_df = pd.read_csv('../Data/spy.csv')
vix_df = pd.read_csv('../Data/vix.csv')
qqq_df = pd.read_csv('../Data/qqq.csv')
iwm_df = pd.read_csv('../Data/iwm.csv')

In [10]:
#Convert Date to datetime
spy_df['Date'] = pd.to_datetime(spy_df['Date'])
vix_df['Date'] = pd.to_datetime(vix_df['Date'])
qqq_df['Date'] = pd.to_datetime(qqq_df['Date'])
iwm_df['Date'] = pd.to_datetime(iwm_df['Date'])

In [11]:
#Set the Date as the index
spy_df.set_index('Date', inplace=True)
vix_df.set_index('Date', inplace=True)
qqq_df.set_index('Date', inplace=True)
iwm_df.set_index('Date', inplace=True)

In [12]:
#Check for null values
print(' SPY \n',spy_df.isna().sum())
print(' VIX \n',vix_df.isna().sum())
print('QQQ \n', qqq_df.isna().sum())
print('IWM \n', iwm_df.isna().sum())


 SPY 
 Adj Close    0
dtype: int64
 VIX 
 Adj Close    0
dtype: int64
QQQ 
 Adj Close    0
dtype: int64
IWM 
 Adj Close    0
dtype: int64


There are no null values which is expected due to the data source

Now we will export all of these seperate dataframes to pickles

In [13]:

spy_df.to_pickle('../Data/spy_df')
vix_df.to_pickle('../Data/vix_df')
qqq_df.to_pickle('../Data/qqq_df')
iwm_df.to_pickle('../Data/iwm_df')

### Clean Sentiment Data

In [79]:
#Read in 3 sentiment csv files
AAII_df = pd.read_csv('../Data/AAII.csv')
NAAIM_df = pd.read_csv('../Data/NAAIM.csv')


In [80]:
#Convert Date to datetime format
AAII_df['Date'] = pd.to_datetime(AAII_df['Date'])

In [81]:
#Convert Date to datetime format
NAAIM_df['Date'] = pd.to_datetime(NAAIM_df['Date'])

I can see that moving the date of the NAAIM data forward by one will result in all the dates being in alignment

In [82]:
NAAIM_df['Date'] = NAAIM_df['Date'] + pd.Timedelta('1 day')

I need to adjust the first NAAIM date back two days to get them in line. After that everything is set

In [83]:

NAAIM_df['Date'].loc[0] = NAAIM_df['Date'].loc[0] - pd.Timedelta('2 days')

Checking that it worked

In [84]:
NAAIM_df['Date'].loc[0]

Timestamp('2006-06-29 00:00:00')

In [85]:
#Set the Date as the index
AAII_df.set_index('Date', inplace=True)
NAAIM_df.set_index('Date', inplace=True)


In [86]:
#Check for null values
print(' AAII \n',AAII_df.isna().sum())
print(' NAAIM \n',NAAIM_df.isna().sum())


 AAII 
 Bullish             0
Neutral             0
Bearish             0
Bull-Bear Spread    0
dtype: int64
 NAAIM 
 Mean/Average             0
Most Bearish Response    0
Quart 2 (median)         0
Most Bullish Response    0
dtype: int64


In [87]:
#Rename columns
AAII_df.rename(columns={"Bull-Bear Spread": "Spread"},inplace=True)

NAAIM_df.rename(columns={"Mean/Average": "Mean","Quart 2 (median)": "Median","Most Bearish Response": "MaxBear","Most Bullish Response": "MaxBull"},inplace=True)


In [88]:
sentiment_df = pd.concat([AAII_df,NAAIM_df],axis=1,join='outer')

In [22]:

sentiment_df


,Bullish,Neutral,Bearish,Spread,Mean,MaxBear,Median,MaxBull
Date,,,,,,,,
2006-06-29,0.386000,0.216400,0.397700,-0.011700,56.33,-125.0,65.0,125.0
2006-07-06,0.377000,0.196700,0.426200,-0.049200,19.44,-100.0,20.0,100.0
2006-07-13,0.365000,0.240900,0.394200,-0.029200,31.20,-50.0,25.0,150.0
2006-07-20,0.238500,0.183500,0.578000,-0.339500,18.76,-100.0,25.0,100.0
2006-07-27,0.348800,0.220900,0.430200,-0.081400,17.38,-50.0,15.0,100.0
...,...,...,...,...,...,...,...,...
2020-04-02,0.342391,0.160326,0.497283,-0.154892,23.67,-100.0,20.5,100.0
2020-04-09,0.365994,0.187320,0.446686,-0.080692,26.74,-100.0,25.0,100.0
2020-04-16,0.348601,0.223919,0.427481,-0.078880,28.71,-100.0,35.0,100.0


I need to drop the first 4 entries of the dataframe

In [93]:
sentiment_df = sentiment_df.iloc[4:]

In [94]:
sentiment_df

,Bullish,Neutral,Bearish,Spread,Mean,MaxBear,Median,MaxBull
Date,,,,,,,,
2006-06-29,0.386000,0.216400,0.397700,-0.011700,56.33,-125.0,65.0,125.0
2006-07-06,0.377000,0.196700,0.426200,-0.049200,19.44,-100.0,20.0,100.0
2006-07-13,0.365000,0.240900,0.394200,-0.029200,31.20,-50.0,25.0,150.0
2006-07-20,0.238500,0.183500,0.578000,-0.339500,18.76,-100.0,25.0,100.0
2006-07-27,0.348800,0.220900,0.430200,-0.081400,17.38,-50.0,15.0,100.0
...,...,...,...,...,...,...,...,...
2020-04-02,0.342391,0.160326,0.497283,-0.154892,23.67,-100.0,20.5,100.0
2020-04-09,0.365994,0.187320,0.446686,-0.080692,26.74,-100.0,25.0,100.0
2020-04-16,0.348601,0.223919,0.427481,-0.078880,28.71,-100.0,35.0,100.0


In [95]:
#Exporting the sentiment dataframe to a pickle
sentiment_df.to_pickle('../Data/sentiment_df')

## Pre-processing

### Sentiment and Stock Data
As the sentiment data is released weekly and there tends to be a significant amount of daily variation in stock market data I'm going to take the weekly average closing price. I'm also going to align the date index for the data

In [14]:
spy_df = pd.read_pickle('../Data/spy_df')
vix_df = pd.read_pickle('../Data/vix_df')
qqq_df = pd.read_pickle('../Data/qqq_df')
iwm_df = pd.read_pickle('../Data/iwm_df')

In [15]:
spy_df=spy_df.resample('W').mean()
vix_df=vix_df.resample('W').mean()
qqq_df=qqq_df.resample('W').mean()
iwm_df=iwm_df.resample('W').mean()

In [16]:
spy_df

,Adj Close
Date,
2006-07-02,96.124527
2006-07-09,96.086758
2006-07-16,94.830073
2006-07-23,93.916246
2006-07-30,95.820935
...,...
2020-04-12,270.555000
2020-04-19,280.590009
2020-04-26,279.156000


In [17]:
vix_df

,Adj Close
Date,
2006-07-02,13.080000
2006-07-09,13.705000
2006-07-16,15.498000
2006-07-23,17.108000
2006-07-30,14.744000
...,...
2020-04-12,44.240000
2020-04-19,39.606000
2020-04-26,41.706001


In [18]:
qqq_df

,Adj Close
Date,
2006-07-02,34.363014
2006-07-09,33.891038
2006-07-16,32.648847
2006-07-23,32.010687
2006-07-30,32.453851
...,...
2020-04-12,198.577500
2020-04-19,210.571997
2020-04-26,210.592001


In [19]:
iwm_df

,Adj Close
Date,
2006-07-02,59.240292
2006-07-09,59.099878
2006-07-16,57.294521
2006-07-23,56.045783
2006-07-30,56.942692
...,...
2020-04-12,116.834999
2020-04-19,119.990001
2020-04-26,120.056000


Align the stock data index with the sentiment data

In [20]:
spy_df.index = spy_df.index - DateOffset(days=3)
vix_df.index = vix_df.index - DateOffset(days=3)
qqq_df.index = qqq_df.index - DateOffset(days=3)
iwm_df.index = iwm_df.index - DateOffset(days=3)

In [21]:
#Add identifier for tickers
spy_df['Ticker']= 'SPY'
vix_df['Ticker']= 'VIX'
qqq_df['Ticker']= 'QQQ'
iwm_df['Ticker']= 'IWM'

In [22]:
#Create a Combined Ticker DF and export all to pickles
ticker_df = pd.concat([spy_df,vix_df,qqq_df,iwm_df],axis=1,join='inner')

In [23]:
ticker_df

,Adj Close,Ticker,Adj Close,Ticker,Adj Close,Ticker,Adj Close,Ticker
Date,,,,,,,,
2006-06-29,96.124527,SPY,13.080000,VIX,34.363014,QQQ,59.240292,IWM
2006-07-06,96.086758,SPY,13.705000,VIX,33.891038,QQQ,59.099878,IWM
2006-07-13,94.830073,SPY,15.498000,VIX,32.648847,QQQ,57.294521,IWM
2006-07-20,93.916246,SPY,17.108000,VIX,32.010687,QQQ,56.045783,IWM
2006-07-27,95.820935,SPY,14.744000,VIX,32.453851,QQQ,56.942692,IWM
...,...,...,...,...,...,...,...,...
2020-04-09,270.555000,SPY,44.240000,VIX,198.577500,QQQ,116.834999,IWM
2020-04-16,280.590009,SPY,39.606000,VIX,210.571997,QQQ,119.990001,IWM
2020-04-23,279.156000,SPY,41.706001,VIX,210.592001,QQQ,120.056000,IWM


There are some dates missing from the stock data, for these ~20 data points I'm going to drop the values from the sentiment dataframe.

In [24]:
ticker_df=ticker_df[:-1]
spy_df=spy_df[:-1]
vix_df=vix_df[:-1]
qqq_df=qqq_df[:-1]
iwm_df=iwm_df[:-1]

In [25]:
sentiment_df = pd.read_pickle('../Data/sentiment_df')

In [26]:
#Create lists of the index differences
stock_date_list=list(ticker_df.index)
sentiment_date_list=list(sentiment_df.index)
diff_list = np.setdiff1d(sentiment_date_list,stock_date_list)
diff_list

array([], dtype=object)

In [27]:
#Drop the rows from the sentiment dataframe that dont exist in the stock data
sentiment_df.drop(diff_list,inplace=True)

In [28]:
#Check that there are no differences left
stock_date_list=list(ticker_df.index)
sentiment_date_list=list(sentiment_df.index)
diff_list = np.setdiff1d(sentiment_date_list,stock_date_list)
diff_list

array([], dtype=object)

In [29]:
ticker_df.shape

(723, 8)

In [30]:
sentiment_df.shape

(723, 8)

In [31]:
#Checking for missing values
sentiment_df.isnull().sum()

Bullish    0
Neutral    0
Bearish    0
Spread     0
Mean       0
MaxBear    0
Median     0
MaxBull    0
dtype: int64

In [32]:
#There are only a small number missing so will simply forward fill
sentiment_df.fillna(method='ffill',inplace=True)
sentiment_df.isnull().sum()

Bullish    0
Neutral    0
Bearish    0
Spread     0
Mean       0
MaxBear    0
Median     0
MaxBull    0
dtype: int64

In [33]:
#Export to pickle
ticker_df.to_pickle('../Data/ticker_df')
spy_df.to_pickle('../Data/spy_df')
vix_df.to_pickle('../Data/vix_df')
qqq_df.to_pickle('../Data/qqq_df')
iwm_df.to_pickle('../Data/iwm_df')
sentiment_df.to_pickle('../Data/sentiment_df')

### New York Times Article Data
Here I will remove stop words from my NYT Data as well as tokenize and lemmatize the data before grouping into weekly time periods.

In [13]:
wn = WordNetLemmatizer()
# Create stop words
stop_words_ = set(stopwords.words('english'))

#Add filter keywords to stop words
more_stop_words = ['stock','market','finance','business','price','debt','portfolio','SP500',
            'nasdaq','dow jones']

stop_words_.update(more_stop_words)



In [14]:
def black_txt(token):
    '''
    This function removes tokens that are in our stop words and punctuation
    
    Parameters:
        Token - words for processing
        
    Return:
        Words without stopwords
    '''
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    '''
    This function removes unnecessary characters and lemmatizes the strings
    
    Parameters:
        Text - words for processing
        
    Return:
        Clean text
    '''
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [15]:
#read in nyt pickle
nyt_df= pd.read_pickle('../Data/nyt_df')
nyt_df

,Headline,Snippet
Date,,
2006-06-30,aventine may late market,share aventine renewable energy hold large eth...
2006-06-30,ethanol outfit offer stock,plant nutrient ethanol concern andersons file ...
2006-06-30,london stock exchange welcome bollywood player,tuesday next week eros international well know...
2006-06-30,house assail media report track finance,largely party line vote lawmakers call news me...
2006-06-30,world business brief europe russia convertible...,russian ruble move step closer become fully co...
...,...,...
2020-04-02,slum epicenter duterte drug crackdown fear lov...,height president rodrigo duterte anti crime ca...
2020-04-21,mortgage firm get reprieve pay investors,federal house finance agency say mortgage serv...
2020-04-21,liu ouqing civil servant help fee wuhan die,china open centrally plan economy liu party se...


In [16]:
#Cleanning and lemmatizing the headlines and snippets
nyt_df['Headline'] = nyt_df['Headline'].apply(clean_txt)
nyt_df['Snippet'] = nyt_df['Snippet'].apply(clean_txt)

In [17]:
nyt_df

,Headline,Snippet
Date,,
2006-06-30,aventine may late,share aventine renewable energy hold large eth...
2006-06-30,ethanol outfit offer,plant nutrient ethanol concern andersons file ...
2006-06-30,london exchange welcome bollywood player,tuesday next week eros international well know...
2006-06-30,house assail media report track,largely party line vote lawmakers call news me...
2006-06-30,world brief europe russia convertible status n...,russian ruble move step closer become fully co...
...,...,...
2020-04-02,slum epicenter duterte drug crackdown fear lov...,height president rodrigo duterte anti crime ca...
2020-04-21,mortgage firm get reprieve pay investors,federal house agency say mortgage servicers mu...
2020-04-21,liu ouqing civil servant help fee wuhan die,china open centrally plan economy liu party se...


In [9]:
#there were some additional duplicates now that the text has been cleaned so dropping those
nyt_df.drop_duplicates(inplace=True)

In [10]:
nyt_df

,Headline,Snippet
Date,,
2006-06-01,glass makers fin price fix,european commission issue one largest fin cart...
2006-06-01,new captain economy face volatile global market,one reach pinnacle wealth prestige dealmaker w...
2006-06-01,google bad business,google wednesday pour cold water speculation m...
2006-06-01,nasdaqs greifeld play tortoise talk,nasdaqs chief executive robert greifeld clearl...
2006-06-01,glass makers fin price fix,european commission issue one largest fin cart...
...,...,...
2020-05-06,get,stone clapboard home stockton adobe house sant...
2020-05-06,home new jersey new mexico wyoming,stone clapboard home stockton adobe house sant...
2020-05-06,plastic shield homemade lysol grocery persevere,coronavirus upend mundane routine city fresh m...


The final step in preprocessing is to remove articles before our start date and after our end date

In [18]:
start_date = '06-29-2006'
end_date = '04-30-2020'

In [19]:

nyt_df=nyt_df.loc[(nyt_df.index > start_date) & (nyt_df.index <= end_date)]

In [20]:
nyt_df

,Headline,Snippet
Date,,
2006-06-30,aventine may late,share aventine renewable energy hold large eth...
2006-06-30,ethanol outfit offer,plant nutrient ethanol concern andersons file ...
2006-06-30,london exchange welcome bollywood player,tuesday next week eros international well know...
2006-06-30,house assail media report track,largely party line vote lawmakers call news me...
2006-06-30,world brief europe russia convertible status n...,russian ruble move step closer become fully co...
...,...,...
2020-04-02,slum epicenter duterte drug crackdown fear lov...,height president rodrigo duterte anti crime ca...
2020-04-21,mortgage firm get reprieve pay investors,federal house agency say mortgage servicers mu...
2020-04-21,liu ouqing civil servant help fee wuhan die,china open centrally plan economy liu party se...


In [21]:
nyt_df.to_pickle('../Data/nyt_df')

Now that the numerical data has been cleaned and the text-based data lemmatized I will move on to some EDA in the next notebook. There will still be a slight amount of processing to do during the sentiment analysis stage.